Importing required libraries

In [1]:
import torch
import sklearn
import torch.nn as nn
import torchmetrics
from torch.optim import Adam, SGD
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd

Reading in the data

In [2]:
data = pd.read_csv("ML_data2.csv",header=None)
data
data = data.to_numpy()

Setting the device for torch

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


Splitting the data into train (50%), test (25%), and validation (25%)

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_valtest, y_train, y_valtest = train_test_split(data[:,:-1], data[:,-1],
                                                                                  train_size=0.5, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_valtest, y_valtest, train_size=0.5, shuffle=False)

For classification, converting the values in the amount overspent into 0 if it is underspent or 1 if it is overspent

In [5]:
y_train = np.where(y_train<=0,0,1)
y_val = np.where(y_val<=0,0,1)
y_test = np.where(y_test<=0,0,1)

Generating torch datasets from the data to be able to easily batch them

In [6]:
from torch.utils.data import TensorDataset



X_train_tensor = TensorDataset(torch.Tensor(X_train),torch.Tensor(y_train))
X_val_tensor = TensorDataset(torch.Tensor(X_val),torch.Tensor(y_val))
X_test_tensor = TensorDataset(torch.Tensor(X_test),torch.Tensor(y_test))

Generating the dataloaders

In [7]:
# creating dataloaders for the datasets

# arguments for the dataloaders
kwargs = {'num_workers': 1, 'pin_memory': True}

batch_size = 100
# dataloaders
train_loader = DataLoader(dataset=X_train_tensor, batch_size=batch_size, shuffle=False, **kwargs)
valid_loader = DataLoader(dataset=X_val_tensor, batch_size=batch_size, shuffle=False, **kwargs)
test_loader  = DataLoader(dataset=X_test_tensor ,  batch_size=batch_size, shuffle=False,  **kwargs)

Testing dataloaders

In [8]:
batch_x, batch_y = next(iter(train_loader))
print(batch_x.shape, batch_y.shape)

torch.Size([100, 4]) torch.Size([100])


Neural Network class with 3 dense layers, and a sigmoid activation layer at the end to get values between 0-1

In [10]:
class Network(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super().__init__()
        self.linear1 = nn.Linear(input_size, hidden_size1)
        self.linear2 = nn.Linear(hidden_size1, hidden_size2)
        self.linear3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = torch.sigmoid(self.linear3(x))
        return x


Training setup

In [11]:
hidden_dim_1 = 64
hidden_dim_2 = 64

lr = 1e-3

epochs = 150

network = Network(4,hidden_dim_1,hidden_dim_2,1)

loss_function = F.binary_cross_entropy
optimizer = Adam(network.parameters(), lr=lr)


Training Loop

In [14]:
network.train()

print("training network...")
for epoch in range(epochs):
    train_overall_loss = 0

    valid_overall_loss = 0

    for batch_idx, (x, y) in enumerate(train_loader):
        x = x.to(device)

        optimizer.zero_grad()

        out = network(x)
        loss = loss_function(out,y.reshape(-1,1))

        train_overall_loss += loss.item()

        loss.backward()
        optimizer.step()

    # validation
    with torch.no_grad():
      for valid_batch_idx, (valid_x, valid_y) in enumerate(valid_loader):
        valid_x = valid_x.to(device)

        # optimizer.zero_grad()

        valid_out = network(valid_x)
        valid_loss = loss_function(valid_out, valid_y.reshape(-1,1))

        valid_overall_loss += valid_loss.item()

    train_n_datapoints = batch_idx * batch_size
    valid_n_datapoints = valid_batch_idx * batch_size

    print("\tEpoch", epoch + 1, " train\tAverage Loss: ", train_overall_loss / train_n_datapoints)
    print("\tEpoch", epoch + 1, " valid\tAverage Loss: ", valid_overall_loss / valid_n_datapoints)

print("Training complete!")
# to save the model, uncomment the line below.
# torch.save(network,"./network.pt")

	Epoch 2  train	Average Loss:  0.007502219591821943
	Epoch 2  valid	Average Loss:  0.007337974835844601
	Epoch 3  train	Average Loss:  0.007460681055273329
	Epoch 3  valid	Average Loss:  0.007630708918851965
	Epoch 4  train	Average Loss:  0.011125243033681597
	Epoch 4  valid	Average Loss:  0.006658045053482056
	Epoch 5  train	Average Loss:  0.010453290368829455
	Epoch 5  valid	Average Loss:  0.00801768702619216
	Epoch 6  train	Average Loss:  0.00886070201226643
	Epoch 6  valid	Average Loss:  0.0067816795671687405
	Epoch 7  train	Average Loss:  0.009930590569972992
	Epoch 7  valid	Average Loss:  0.012839403853696935
	Epoch 8  train	Average Loss:  0.016574992162840706
	Epoch 8  valid	Average Loss:  0.016105057632221895
	Epoch 9  train	Average Loss:  0.009384420667375838
	Epoch 9  valid	Average Loss:  0.009288057790083043
	Epoch 10  train	Average Loss:  0.0079748290010861
	Epoch 10  valid	Average Loss:  0.007082099704181446
	Epoch 11  train	Average Loss:  0.0067494869572775704
	Epoch 11  

Testing on the test set

In [31]:
network.eval()

x_original_list = []
y_list = []
out_list = []

with torch.no_grad():
    for batch_idx, (test_x, test_y) in enumerate(tqdm(test_loader)):
        test_x = test_x.to(device)

        test_out = network(test_x)
        x_original_list.append(test_x)
        y_list.append(test_y)
        out_list.append(test_out)


100%|██████████| 18/18 [00:00<00:00, 20.10it/s]


Converting output and the target (y) into 1D tensors

In [61]:
y_tensor = torch.Tensor()
out_tensor = torch.Tensor()

for i in range(len(out_list)):
    out_tensor = torch.cat((out_tensor,out_list[i]))
    y_tensor = torch.cat((y_tensor,y_list[i]))

out_tensor = torch.where(out_tensor<0.5,0,1).reshape((-1,))
y_tensor = y_tensor.reshape((-1,))
print(out_tensor.shape)
print(y_tensor.shape)

torch.Size([1767])
torch.Size([1767])


F1 score calculation

In [62]:
f1 = torchmetrics.F1Score(task="binary",num_classes=2)
score = f1(out_tensor,y_tensor)
print(score)

tensor(0.0050)


Confusion Matrix

In [63]:
confusion = torchmetrics.ConfusionMatrix(task="binary",num_classes=2)
conf_matrix = confusion(out_tensor,y_tensor)
print(conf_matrix)

tensor([[1371,   20],
        [ 375,    1]])


Accuracy

In [64]:
acc = torchmetrics.Accuracy(task="binary",num_classes=2)
accuracy = acc(out_tensor,y_tensor)
print(accuracy)

tensor(0.7765)
